### dynamoDB
This notebook adds data to the dynamoDB database

In [1]:
import boto3
import os
import json
import csv
from dotenv import load_dotenv
import numpy as np
import pandas as pd

In [2]:
# Load the environment variables from the .env file
load_dotenv()

True

In [3]:
# Creating the DynamoDB Client
# make sure to set the AWS_ACCESS_KEY_ID and
# AWS_SECRET_ACCESS_KEY environment variables
# OR pass credentials when creating the client:
dynamodb_client = boto3.client('dynamodb', region_name="us-east-1")

TABLE_NAME = "fumehoods"

dyn_resource = boto3.resource("dynamodb", region_name="us-east-1")
TABLE_NAME = "fumehoods"
table = dyn_resource.Table(TABLE_NAME)

In [55]:
def format_item(row):
    formatted_row = {}
    for key, value in row.items():
        if type(value) == str:
            formatted_row[key] = {'S': str(value)}
        if type(value) == int:
            formatted_row[key] = {'N': value}
    return {'M':formatted_row}

def read_fumehood_csv(path="./Point Names/biotech point names.csv"):
    names_df = pd.read_csv(path)
    names_df['name'] = names_df['building'].str.capitalize() + \
                            ".Floor_" + names_df['floor'].astype(str) + \
                            ".Lab_" + names_df['lab'] + \
                            ".Hood_" + names_df['hood'].astype(str)
    return names_df

def get_buildings_dict(buildings_columns = ["external_temp_sensor"]):
    buildings_names_df = read_fumehood_csv().copy().groupby('building').first().reset_index()
    buildings_names_df['building_name'] = buildings_names_df['building'].str.capitalize()
    buildings_names_df = buildings_names_df.set_index('building_name')[buildings_columns]

    buildings = {
                "id": "buildings",
                "map": buildings_names_df.to_dict()
    }

    return buildings

def get_labs_dict(labs_columns = ["building", "floor", "lab", "room_occ_sensor", "internal_temp_sensor"]):
    labs_names_df = read_fumehood_csv().copy().groupby(['building','floor','lab']).first().reset_index()
    labs_names_df['lab_name'] = labs_names_df['building'].str.capitalize() + \
                                        ".Floor_" + labs_names_df['floor'].astype(str) + \
                                        ".Lab_" + labs_names_df['lab'].str.lower()
    labs_names_df = labs_names_df.set_index('lab_name')[labs_columns]

    # TODO: Hardcoded
    labs_names_df.rename(index={"Biotech.Floor_G.Lab_g54":"Biotech.GroundFloor.Lab_g54"}, inplace=True)

    count = read_fumehood_csv().groupby(['building','floor','lab'])['hood'].count()
    count.name = 'hood_count'

    labs_names_df = labs_names_df.merge(count, left_on=['building','floor','lab'], right_index=True)

    labs = {
        "id": "labs",
        "map": labs_names_df.to_dict('index')
    }

    return labs

def get_hoods_dict(hoods_columns = ["building", "floor", "lab", "hood", "flow_sensor","sash_position_sensor"]):
    hoods_names_df = read_fumehood_csv().set_index('name')[hoods_columns]

    #hoods_data = hoods_names_df.apply(format_item, axis=1).to_dict()

    hoods = {
                "id": "hoods",
                "map": hoods_names_df.to_dict('index')
    }

    return hoods

In [32]:
get_hoods_dict()

{'id': 'hoods',
 'map': {'Biotech.Floor_G.Lab_G54.Hood_1': {'building': 'biotech',
   'floor': 'G',
   'lab': 'G54',
   'hood': 1,
   'flow_sensor': '#biotech/biotech_ground_floor/ground_floor_fume_hood_lab_spaces/lab_g54_control/hoodvalve_flow',
   'sash_position_sensor': '#biotech/biotech_ground_floor/ground_floor_fume_hood_lab_spaces/lab_g54_control/hood_sash'},
  'Biotech.Floor_1.Lab_141.Hood_1': {'building': 'biotech',
   'floor': '1',
   'lab': '141',
   'hood': 1,
   'flow_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_141_control/hoodvalve_flow',
   'sash_position_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_141_control/hood_sash'},
  'Biotech.Floor_1.Lab_143.Hood_1': {'building': 'biotech',
   'floor': '1',
   'lab': '143',
   'hood': 1,
   'flow_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_143_control/hoodvalve_flow',
   'sash_position_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hoo

In [56]:
with table.batch_writer() as writer:
    writer.put_item(Item=get_buildings_dict())

response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "buildings"}}
)
print("buildings:")
display(response["Item"])

buildings:


{'id': {'S': 'buildings'},
 'map': {'M': {'external_temp_sensor': {'M': {'Biotech': {'S': '#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat'}}}}}}

In [22]:
with table.batch_writer() as writer:
    writer.put_item(Item=get_labs_dict())

response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "labs"}}
)
print("Labs:")
display(response["Item"])

Labs:


{'id': {'S': 'labs'},
 'map': {'M': {'Biotech.Floor_2.Lab_203b': {'M': {'room_occ_sensor': {'S': '#biotech_lab_203b_hvac/occ_sensor'},
     'internal_temp_sensor': {'S': '#biotech_lab_203b_hvac/zone_temp'},
     'hood_count': {'N': '1'},
     'floor': {'S': '2'},
     'lab': {'S': '203B'},
     'building': {'S': 'biotech'}}},
   'Biotech.Floor_2.Lab_227b': {'M': {'room_occ_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_227b_control/occ'},
     'internal_temp_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_227b_control/zone/zone_temp/trend_log'},
     'hood_count': {'N': '1'},
     'floor': {'S': '2'},
     'lab': {'S': '227B'},
     'building': {'S': 'biotech'}}},
   'Biotech.Floor_3.Lab_327': {'M': {'room_occ_sensor': {'S': '#biotech/biotech_3rd_floor/third_floor_fume_hood_lab_spaces/lab_327_control/occ'},
     'internal_temp_sensor': {'S': '#biotech/biotech_3rd_floor/third_floor_fume_hood_lab_spaces/lab_327_control/zo

In [33]:
with table.batch_writer() as writer:
    writer.put_item(Item=get_hoods_dict())

response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "hoods"}}
)
print("Hoods:")
display(response["Item"])

Hoods:


{'id': {'S': 'hoods'},
 'map': {'M': {'Biotech.Floor_2.Lab_257.Hood_1': {'M': {'flow_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_257_control/hoodvalve_flow'},
     'sash_position_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_257_control/hood_sash'},
     'floor': {'S': '2'},
     'lab': {'S': '257'},
     'hood': {'N': '1'},
     'building': {'S': 'biotech'}}},
   'Biotech.Floor_2.Lab_253B.Hood_1': {'M': {'flow_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_253b_control/hoodvalve_flow'},
     'sash_position_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_253b_control/hood_sash'},
     'floor': {'S': '2'},
     'lab': {'S': '253B'},
     'hood': {'N': '1'},
     'building': {'S': 'biotech'}}},
   'Biotech.Floor_4.Lab_417B.Hood_1': {'M': {'flow_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417b_control/hoodvalve_flow

In [52]:
dynamodb = boto3.resource('dynamodb', region_name="us-east-1")

table = dynamodb.Table('fumehoods')

response = table.scan(ProjectionExpression="id")
response['Items']

[{'id': 'buildings'}, {'id': 'occupants'}, {'id': 'hoods'}, {'id': 'labs'}]

In [51]:
table.delete_item(
    Key={
        'id': 'maggie'
    }
)

{'ResponseMetadata': {'RequestId': '1GHKH59R53SLGJ5RFIAD0MDMA7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 09 Nov 2024 05:30:16 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1GHKH59R53SLGJ5RFIAD0MDMA7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [12]:
# ONLY USE THIS IF WE'RE GOING TO DO THE DYNAMODB APPROACH FOR STORING RANKINGS!!!!

# response = dynamodb_client.get_item(
#     TableName=TABLE_NAME, Key={"id": {"S": "labs"}}
# )

# labs_dict = response['Item']["map"]["M"]

# df = pd.DataFrame.from_dict({(i, j): labs_dict[i][j] 
#                               for i in labs_dict.keys() 
#                               for j in labs_dict[i].keys()},
#                              orient='index')

# df.index = df.index.droplevel(1)

# # Reorder columns
# df = df[['building', 'floor', 'lab', 'internal_temp_sensor', 'week_sash_time', 'year_sash_time', 'occ_sensor', 'day_sash_time', 'month_sash_time']]

# labs_df = df.applymap(lambda x: list(x.values())[0])

# labs_df = labs_df.sort_values(by="day_sash_time")

# labs_df["Ranking"] = np.arange(1, len(labs_df) + 1)
# labs_df['Ranking_Emoji'] = labs_df['Ranking'].copy()
# labs_df.loc[labs_df['Ranking']==1, 'Ranking_Emoji'] = "🥇"
# labs_df.loc[labs_df['Ranking']==2, 'Ranking_Emoji'] = "🥈"
# labs_df.loc[labs_df['Ranking']==3, 'Ranking_Emoji'] = "🥉"

# labs_df

# # labs_dynamo_biotech = {k:v for (k,v) in labs_dict.items() if "Biotech" in k}
# # labs_dynamo_biotech

,building,floor,lab,internal_temp_sensor,week_sash_time,year_sash_time,occ_sensor,day_sash_time,month_sash_time,Ranking,Ranking_Emoji
Biotech.Floor_4.Lab_417B,biotech,4,418,#biotech/biotech_4th_floor/fourth_floor_fume_h...,0,0,#biotech/biotech_4th_floor/fourth_floor_fume_h...,0,0,1,🥇
Biotech.Floor_3.Lab_303,biotech,3,303B,#biotech/biotech_3rd_floor/third_floor_fume_ho...,0,0,#biotech/biotech_3rd_floor/third_floor_fume_ho...,0,0,2,🥈
Biotech.Floor_4.Lab_427B,biotech,4,433,#biotech/biotech_4th_floor/fourth_floor_fume_h...,0,0,#biotech/biotech_4th_floor/fourth_floor_fume_h...,0,0,3,🥉
Biotech.Floor_3.Lab_360B,biotech,3,367,#biotech/biotech_3rd_floor/third_floor_fume_ho...,0,0,#biotech/biotech_3rd_floor/third_floor_fume_ho...,0,0,4,4
Biotech.Floor_3.Lab_341,biotech,3,341B,#biotech/biotech_3rd_floor/third_floor_fume_ho...,0,0,#biotech/biotech_3rd_floor/third_floor_fume_ho...,0,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...
Biotech.Floor_4.Lab_457,biotech,4,457B,#biotech/biotech_4th_floor/fourth_floor_fume_h...,0,0,#biotech/biotech_4th_floor/fourth_floor_fume_h...,0,0,62,62
Biotech.Floor_4.Lab_417,biotech,4,417B,#biotech/biotech_4th_floor/fourth_floor_fume_h...,0,0,#biotech/biotech_4th_floor/fourth_floor_fume_h...,0,0,63,63
Biotech.Floor_1.Lab_160,biotech,2,217B,#biotech/biotech_2nd_floor/second_floor_fume_h...,0,0,#biotech/biotech_2nd_floor/second_floor_fume_h...,0,0,64,64
Biotech.Floor_4.Lab_421,biotech,4,421B,#biotech/biotech_4th_floor/fourth_floor_fume_h...,0,0,#biotech/biotech_4th_floor/fourth_floor_fume_h...,0,0,65,65


In [ ]:
# read Robbie's csv output here
with open('namingconvention.csv', 'r') as file:
    csv_dict_reader = csv.DictReader(file)
    for row in csv_dict_reader:


        hoods = {
            "id": "hoods",
            "map": {
                row['name']: { #"Biotech.Floor_3.Lab_317.Hood_1"
                    "building": row['building'], #"Biotech"
                    "floor": row['Floor'], #"3"
                    "lab": row['Lab'], # "317"
                    "hood": row['hood'], # "1"
                    "flow_sensor": row['Flow Sensor'], # "#path_to_this_sensor"
                    "sash_sensor": row['Sash'], # "#path_to_this_sensor"
                },
                # "Biotech.Floor_4.Lab_433.Hood_1": {
                #     "building": "Biotech",
                #     "floor": "4",
                #     "lab": "433",
                #     "hood": "1",
                #     "flow_sensor": "#path_to_this_sensor",
                #     "sash_sensor": "#path_to_this_sensor",
                # },
            },
        }

        buildings = {
            "id": "buildings",
            "map": {
                # "Biotech": {"external_temp_sensor": "#path_to..."},
                # "Weill": {"external_temp_sensor": "#path_to..."},
                row['name']: {"external_temp_sensor": row['External Temp']}
            },
        }

        labs = {
            "id": "labs",
            "map": {
                row['name']: { #"Biotech.Floor_3.Lab_317"
                    "building": row['building'], #"Biotech"
                    "floor": row['Floor'], # "3"
                    "lab": row['Lab'], # "317"
                    "occ_sensor": row['Occupancy'], # "#path_to_this_sensor"
                    "internal_temp_sensor": row['Internal Temp'], # "#path_to_this_sensor"
                },
            },
        }

        occupants = {
            "id": "occupants",
            "map": {
                "rf377@cornell.edu": {
                    "name": "Reid Fleishman",
                    "lab": "Biotech.Floor_3.Lab_317",
                },
            },
        }

        # Put the above data into the table
        with table.batch_writer() as writer:
            response = writer.put_item(Item=hoods)
            print(response)
            response = writer.put_item(Item=buildings)
            print(response)
            response = writer.put_item(Item=labs)
            print(response)
            response = writer.put_item(Item=occupants)
            print(response)

        # Print what was just added
        response = dynamodb_client.get_item(
            TableName=TABLE_NAME, Key={"id": {"S": "hoods"}}
        )
        print("Hoods:")
        display(response["Item"])

        response = dynamodb_client.get_item(
            TableName=TABLE_NAME, Key={"id": {"S": "buildings"}}
        )
        print("Buildings:")
        display(response["Item"])

        response = dynamodb_client.get_item(
            TableName=TABLE_NAME, Key={"id": {"S": "labs"}}
        )
        print("Labs:")
        display(response["Item"])

        response = dynamodb_client.get_item(
            TableName=TABLE_NAME, Key={"id": {"S": "occupants"}}
        )
        print("Occupants:")
        display(response["Item"])

In [34]:
dynamodb = boto3.resource('dynamodb', region_name="us-east-1")

table = dynamodb.Table('fumehoods')

response = table.scan(ProjectionExpression="id")
data = response['Items']

# Uncomment for large queries
# while 'LastEvaluatedKey' in response:
#     response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
#     data.extend(response['Items'])

data

[{'id': 'Biotech.Floor_4.Lab_433.Hood_1'},
 {'id': 'BiotechnologyBuilding/lab_433/hood'},
 {'id': 'buildings'},
 {'id': 'cat'},
 {'id': 'Biotech.Floor_3.Lab_317.Hood_1'},
 {'id': 'occupants'},
 {'id': 'maggie'},
 {'id': 'Biotech.Floor_4.Lab_433.Hood_2'},
 {'id': 'hoods'},
 {'id': 'labs'},
 {'id': 'Biotech.Floor_4.Lab_441.Hood_1'}]

In [ ]:
response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "labs"}}, ProjectionExpression="#map_alt.#lab", ExpressionAttributeNames={"#map_alt":"map", "#lab":"Biotech.Floor_4.Lab_417"}
)
display(response["Item"])

{'map': {'M': {'Biotech.Floor_4.Lab_417': {'M': {'room_occ_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417_control/occ'},
     'internal_temp_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417_control/zone/zone_temp/trend_log'},
     'hood_count': {'N': '1'},
     'floor': {'S': '4'},
     'lab': {'S': '417'},
     'building': {'S': 'biotech'}}}}}}

In [23]:
response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "hoods"}}, ProjectionExpression="#map_alt.#hood", ExpressionAttributeNames={"#map_alt":"map", "#hood":"Biotech.Floor_4.Lab_417.Hood_1"}
)
display(response["Item"])

{'map': {'M': {'Biotech.Floor_4.Lab_417.Hood_1': {'M': {'flow_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417_control/hoodvalve_flow'},
     'sash_position_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417_control/hood_sash'},
     'floor': {'S': '4'},
     'lab': {'S': '417'},
     'hood': {'N': '1'},
     'building': {'S': 'biotech'}}}}}}